# Auto Sampler script
## Elveflow Instruments
* OB1 pressure controller
* MUX Wire
* 

## Import Library

In [ ]:
# Import Elveflow library
import sys
from email.header import UTF8
SDK_HOME = 'C:/Users/T5810/Downloads/ESI_V3_04_01/SDK'
sys.path.append(SDK_HOME+'/DLL64/DLL64')#add the path of the library here
sys.path.append(SDK_HOME+'/Python_64')#add the path of the LoadElveflow.py

from ctypes import *

from array import array

from Elveflow64 import *

## Instrument Class definitions

In [ ]:
def _check_error(task, error):
    err_dict = {
        -8000: 'No Digital Sensor found',
        -8001: 'No pressure sensor compatible with OB1 MK3',
        -8002: 'No Digital pressure sensor compatible with OB1 MK3+',
        -8003: 'No Digital Flow sensor compatible with OB1 MK3',
        -8004: 'No IPA config for this sensor',
        -8005: 'Sensor not compatible with AF1',
        -8006: 'No Instrument with selected ID'
    }
    if error != 0:
        if error in err_dict:
            raise RuntimeError('{} failed with error code {}\n{}'.format(task, error, err_dict[error]))
        else:
            raise RuntimeError('{} failed with error code {}\n{}'.format(task, error, 'Unknown error, see http://www.ni.com/pdf/manuals/321551a.pdf'))
            
class ob1():
     def __init__(self, calibrate=False):        
            address='01C1690E'
            self.instr_ID = c_int32()
            print('Instrument name and regulator types hardcoded in the python script'.encode('utf-8'))
            # see User guide to determine regulator type NI MAX to determine the instrument name
            # channel 1: -1000~1000 mBar, channel 2: none, channel 3: none, channel 4:none
            error = OB1_Initialization(address.encode('ascii'), 4, 0, 0, 0, byref(self.instr_ID))
            # all functions will return error code to help you to debug your code, for further information see user guide
            _check_error('OB1 Initialization', error)

            # add one digital flow sensor
            error = OB1_Add_Sens(self.instr_ID, 1, 4, 0, 1)
            _check_error('Adding digital flow sensor', error)

            calib_path = os.path.expanduser(r'~\ob1_calibration.txt')
            self.calib = (c_double * 1000)()
            if calibrate:
                print ('Starting calibration')
                OB1_Calib(self.instr_ID.value, self.calib, 1000)
                error = Elveflow_Calibration_Save(calib_path.encode('ascii'), byref(self.calib), 1000)
                print ('Calibration finished')
                print ('Calibration saved in file %s' % calib_path.encode('ascii'))
            else:
                if not os.path.isfile(calib_path):
                    raise IOError('Calibration file "{}" does not exist'.format(calib_path))
                error = Elveflow_Calibration_Load(calib_path.encode('ascii'), byref(self.calib), 1000)
                _check_error('Loading calibration file', error)

    def measure(self, channel=0):
        """
        Measures the instantaneous flow rate, on designated port.
        """
        set_channel = int(channel)  # convert to int
        set_channel = c_int32(channel)  # convert to c_int32
        get_flowrate = c_double()
        error = OB1_Get_Sens_Data(self.instr_ID.value, set_channel, 1, byref(get_pressure))  # Acquire_data =1 -> Read all the analog value
        _check_error('Getting data from flow sensor', error)
        return get_flowrate.value

    def set_pressure(self, pressure, port=0):
        """
        Sets the pressure, on designated port.
        """
        set_channel = int(port)  # convert to int
        set_channel = c_int32(port)  # convert to c_int32
        set_pressure = float(pressure)
        set_pressure = c_double(pressure)  # convert to c_double
        error = OB1_Set_Press(self.instr_ID.value, set_channel, set_pressure, byref(self.calib), 1000)
        _check_error('Setting pressure', error)
        pass
    
    def __del__(self):
        error=OB1_Destructor(self.Instr_ID.value)
        
# OB1 and flow sensor initialization
Instr_ID=c_int32()
print("Instrument name and regulator types are hardcoded in the Python script")
#see User Guide to determine regulator types and NIMAX to determine the instrument name 
error=OB1_Initialization('01C9D9C3'.encode('ascii'),1,2,4,3,byref(Instr_ID)) 
#all functions will return error codes to help you to debug your code, for further information refer to User Guide
print('error:%d' % error)
print("OB1 ID: %d" % Instr_ID.value)

#add one digital flow sensor with water calibration (OB1 MK3+ only), all information to declare sensors are described in the User Guide
error=OB1_Add_Sens(Instr_ID, 1, 1, 1, 0, 7, 0)
#(CustomSens_Voltage_5_to_25 only works with CustomSensors and OB1 from 2020 and after)
print('error add digit flow sensor:%d' % error)


#
#Set the calibration type
#

Calib=(c_double*1000)()#always define array this way, calibration should have 1000 elements
repeat=True
while repeat==True:
    answer=input('select calibration type (default, load, new ) : ')
    #answer='default'#test purpose only
    Calib_path='C:\\Users\\Public\\Desktop\\Calibration\\Calib.txt'
    if answer=='default':
        error=Elveflow_Calibration_Default (byref(Calib),1000)
        #for i in range (0,1000):
            #print('[',i,']: ',Calib[i])
        repeat=False
    if answer=='load':
        error=Elveflow_Calibration_Load (Calib_path.encode('ascii'), byref(Calib), 1000)
        #for i in range (0,1000):
            #print('[',i,']: ',Calib[i])
        repeat=False
        
    if answer=='new':
        OB1_Calib (Instr_ID.value, Calib, 1000)
        #for i in range (0,1000):
            #print('[',i,']: ',Calib[i])
        error=Elveflow_Calibration_Save(Calib_path.encode('ascii'), byref(Calib), 1000)
        print ('calib saved in %s' % Calib_path.encode('ascii'))
        repeat=False

In [5]:
task = 'test'
error = 1
err_dict = {
        -8000: 'No Digital Sensor found',
        -8001: 'No pressure sensor compatible with OB1 MK3',
        -8002: 'No Digital pressure sensor compatible with OB1 MK3+',
        -8003: 'No Digital Flow sensor compatible with OB1 MK3',
        -8004: 'No IPA config for this sensor',
        -8005: 'Sensor not compatible with AF1',
        -8006: 'No Instrument with selected ID'
    }
if error != 0:
    if error in err_dict:
        raise RuntimeError('{} failed with error code {}\n{}'.format(task, error, err_dict[error]))
    else:
        raise RuntimeError('{} failed with error code {}\n{}'.format(task, error, 'Unknown error, see http://www.ni.com/pdf/manuals/321551a.pdf'))

RuntimeError: test failed with error code 1
Unknown error, see http://www.ni.com/pdf/manuals/321551a.pdf

In [ ]:
time_step